In [0]:
%pip install tqdm

In [0]:
import requests
import pandas as pd
import time
import src.config as config
from pathlib import Path
from tqdm import tqdm
import random
from download_skogsbilveg import hent_skogsbilveier_og_noder

"""
def er_ekte_endepunkt(nodeid):
    url = f"https://nvdbapiles-v3.atlas.vegvesen.no/vegnett/noder/{nodeid}"
    headers = {
        "Accept": "application/vnd.vegvesen.nvdb-v3-rev4+json",
        "X-Client": "Systemet for vegobjekter",
    }
    params = {"srid": "25833"}
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()
    data = response.json()
    porter = data.get("porter", [])
    # time.sleep(0.2)  # For å unngå for mange forespørsler på kort tid
    return len(porter) == 1


def filtrer_ekte_endepunkter(df):
    ekte_rows = []
    for idx, row in df.iterrows():
        nodeid = row["nodeid"]
        if er_ekte_endepunkt(nodeid):
            ekte_rows.append(row)
    return pd.DataFrame(ekte_rows) 
"""

In [0]:
def make_bbox_around_endepunkt(x, y, buffer_x, buffer_y):
    return [x - buffer_x, y - buffer_y, x + buffer_x, y + buffer_y]


def get_wms_url(bbox):
    bbox_str = ",".join(map(str, bbox))
    width, height = config.IMAGE_SIZE
    BASE_IMAGE_URL = config.BASE_IMAGE_URL
    layer = "ortofoto"

    return (
        f"{BASE_IMAGE_URL}?"
        f"SERVICE=WMS&"
        f"VERSION=1.3.0&"
        f"REQUEST=GetMap&"
        f"layers={layer}&"
        f"STYLES=Default&"
        f"CRS=EPSG:25833&"
        f"BBOX={bbox_str}&"
        f"width={width}&"
        f"height={height}&"
        f"FORMAT=image/png"
    )


def download_image_from_wms(wms_url, save_path):
    try:
        response = requests.get(wms_url, timeout=10)
        response.raise_for_status()
        with open(save_path, "wb") as f:
            f.write(response.content)
        print(f"✅ Lagret bilde: {save_path}")
        return True
    except requests.RequestException as e:
        print(f"❌ Feil ved nedlasting: {e}")
        return False


def hent_wkt_koordinater(nodeid, srid="utm33", max_retries=3, timeout=5):
    url = f"https://nvdbapiles-v3.atlas.vegvesen.no/vegnett/noder/{nodeid}"
    headers = {
        "Accept": "application/vnd.vegvesen.nvdb-v3-rev4+json",
        "X-Client": "Systemet for vegobjekter",
    }
    params = {"srid": srid}

    for attempt in range(max_retries):
        try:
            response = requests.get(
                url, headers=headers, params=params, timeout=timeout
            )
            if response.status_code == 404:
                print(f"⚠️ 404 - Ingen node for {nodeid}")
                return False, None, None, None
            response.raise_for_status()

            data = response.json()
            porter = data.get("porter", [])
            er_ekte = (
                len(porter) == 1
                and porter[0].get("tilkobling", {}).get("portnummer") == 1
            )

            wkt = data.get("geometri", {}).get("wkt")
            if wkt and "Z(" in wkt:
                coords = wkt.split("Z(")[1].split(")")[0].split()
                x, y = float(coords[0]), float(coords[1])
            else:
                x, y = None, None

            return er_ekte, wkt, x, y

        except requests.exceptions.RequestException as e:
            print(f"❌ Feil på forsøk {attempt + 1} for nodeid {nodeid}: {e}")
            time.sleep(120)  # pause før nytt forsøk

    print(f"❌ Ga opp etter {max_retries} forsøk for nodeid {nodeid}")
    return False, None, None, None


def filtrer_ekte_endepunkter(df):
    ekte_rows = []
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Henter NVDB noder"):
        nodeid = row["nodeid"]
        er_ekte, wkt, x, y = hent_wkt_koordinater(nodeid)
        if er_ekte:
            ekte_rows.append(
                {
                    "nodeid": nodeid,
                    "wkt": wkt,
                    "x": x,
                    "y": y,
                }
            )
    return pd.DataFrame(ekte_rows, columns=["nodeid", "wkt", "x", "y"])


def main():
    df = hent_skogsbilveier_og_noder("0301")
    print(f"Antall noder totalt: {len(df)}")

    ekte_df = filtrer_ekte_endepunkter(df)
    print(f"Antall ekte endepunkter: {len(ekte_df)}")

    image_dir = (
        "/Volumes/land_topografisk-gdb_dev/external_dev/static_data/DL_SNUPLASSER/test/"
    )
    image_paths = []

    for idx, row in tqdm(
        ekte_df.iterrows(), total=len(ekte_df), desc="Laster ned bilder"
    ):
        x, y = row["x"], row["y"]
        nodeid = row["nodeid"]
        bbox = make_bbox_around_endepunkt(x, y, buffer_x=50, buffer_y=50)
        wms_url = get_wms_url(bbox)
        image_path = image_dir / f"endepunkt_{nodeid}.png"
        success = download_image_from_wms(wms_url, image_path)
        if success:
            image_paths.append(str(image_path))
        else:
            image_paths.append(None)


main()